In [34]:
from openai import OpenAI
import os
import json
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()

# Set parameters

name = "1"
model = "gpt-3.5-turbo"               # important options: "gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"
loops = 1
memory = 1
stable_system_message = False
assistant_transform = False    
assistant_transform_type = "user"     # options: "user", "user-assistant", "system"
stream = False

first_prompt = [
  #{"role": "system", "content": ""},
  {"role": "user", "content": ""}
]

# Choose or create the right folder with json files

folder_name = f"{model} memory-{memory}"
if assistant_transform:
  folder_name += f" {assistant_transform_type}"
os.makedirs(folder_name, exist_ok=True)

for loop in range(loops):

  # Load completions from json, if continuing with a file generated in the past

  try:
    with open(f"{folder_name}/{name}.json", "r") as json_file:
      completions=json.load(json_file)

  # If not, create a new completions database

  except FileNotFoundError:
    completions = []

  # Collect messages generated so far

  messages = first_prompt
  for completion in completions:
      message = completion["choices"][0]["message"]
      messages.append({"role": message["role"], "content": message["content"]}   
      )

  # Pick prompt messages

  if memory >= len(messages):
      prompt_messages = messages
  else:
      #print(f"Memory smaller than the entire conversation. Forgetting {len(messages)-memory} messages")
      prompt_messages = []
      if stable_system_message:
        prompt_messages.append(first_prompt[0])
        print(prompt_messages)
        print("stable_system_message")
      prompt_messages = prompt_messages+messages[(len(messages)-memory):]

  # Transform assistant role to assistant and user, user, or system roles

  if assistant_transform:
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages):
        if prompt_messages[-i]["role"] == "assistant":
          prompt_messages[-i]["role"] = "user"
        i += 2
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type


  # Send to OpenAI API to get a new completion

  completion = client.chat.completions.create(
        model=model,
        messages = prompt_messages,
        stream = stream,
      )
  #print(f"Prompt messages sent to OpenAI API:{prompt_messages}")
  if stream:
    for chunk in stream:
      print(chunk.choices[0].delta.content or "", end="")
  else:
    print(completion["choices"][0]["message"]["content"])

  # Update database and save it to the right json file

  completions.append(completion.model_dump())
  with open(f"{folder_name}/{name}.json", "w") as json_file:
      json.dump(completions,json_file)

 

Prompt messages sent to OpenAI API:[{'role': 'user', 'content': '\n16. The globalization of popular culture through the internet and social media has led to the sharing of ideas, trends, and influences on a global scale. This interconnectedness has allowed for the exchange of cultural practices and traditions, creating a more diverse and inclusive cultural landscape.\n17. Popular culture can also act as a form of escapism for individuals, providing entertainment and distraction from the stresses and challenges of everyday life. Whether through music, movies, or video games, popular culture can serve as a form of entertainment that allows people to relax and unwind.\n18. The rise of influencer culture within popular culture has transformed the way individuals consume content and make purchasing decisions. Influencers have the power to shape trends and influence consumer behavior, blurring the lines between advertising and entertainment.\n19. The phenomenon of viral content within popula

In [42]:
## Display data

# Load the JSON file

file_path = f"{folder_name}/{name}.json"

#file_path = f"gpt-3.5-turbo memory-1 user/2.json"

with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the JSON data in completions

normalized_data = pd.json_normalize(data, record_path='choices', meta=[
    'id', 'created', 'model', 'object', 'service_tier', 'system_fingerprint', 
    ['usage', 'completion_tokens'], ['usage', 'prompt_tokens'], ['usage', 'total_tokens']], sep='.',
    meta_prefix='', record_prefix='choices.')

# Make more important columns more visible

normalized_data = normalized_data[['choices.message.content', 'model', 
       'usage.completion_tokens', 'usage.prompt_tokens', 'usage.total_tokens', 'id', 
       'choices.finish_reason', 'choices.index', 'choices.logprobs', 
       'choices.message.role', 'choices.message.function_call', 'choices.message.tool_calls', 
       'created', 'object', 'service_tier', 'system_fingerprint'
       ]]

# Display

content_all = normalized_data["choices.message.content"]
last = content_all.iloc[-1]

print(file_path)
print(len(content_all))

for content in content_all:
   display_markdown(content, raw=True)

gpt-3.5-turbo memory-1 user/4.json
20


Hello! How can I assist you today?

Hello! I don't really need any assistance at the moment, but I appreciate you asking. How are you doing today?

I'm just a computer program, so I don't have feelings, but thank you for asking! Is there anything you'd like to talk about or ask me?

I mostly answer questions and provide information on a wide range of topics. Feel free to ask me anything you'd like to know, and I'll do my best to help you.

...


36. Popular culture often serves as a form of escapism and entertainment, providing individuals with a temporary refuge from the challenges and stress of everyday life. Whether through movies, TV shows, music, or other forms of media, popular culture offers moments of joy and relief that allow us to recharge and reset.
37. Popular culture has the power to challenge societal norms and stereotypes, promoting diversity and inclusivity in mainstream media. By representing underrepresented groups and highlighting their stories, popular culture can promote empathy, understanding, and acceptance among audiences.
38. Popular culture can act as a catalyst for social change by raising awareness of important issues and inspiring individuals to take action. Through powerful storytelling and compelling narratives, popular culture can mobilize communities and drive positive change in the world.
39. The globalization of popular culture has created opportunities for cross-cultural exchange and understanding, as individuals from different parts of the world can connect over shared interests and experiences. Through the dissemination of popular cultural products, people can learn about and appreciate diverse cultures and perspectives.
40. Ultimately, popular culture is a dynamic and ever-evolving force that shapes our society and influences our individual and collective identities. By engaging with popular culture critically and reflectively, we can harness its power to create positive change and foster a more inclusive and empathetic world.


41. Popular culture can also play a role in shaping our personal preferences, values, and beliefs. The media we consume and the celebrities we admire can influence our attitudes towards various issues, shaping our cultural and social identity.
42. In addition, popular culture can serve as a platform for self-expression and creativity, allowing individuals to explore their own identities and share their unique perspectives with the world. Whether through art, fashion, or social media, popular culture provides a space for individuals to express themselves and connect with like-minded individuals.
43. Popular culture can also have a significant impact on the economy, as industries such as film, music, fashion, and technology generate billions of dollars in revenue each year. The popularity of certain trends and products in popular culture can drive consumer behavior and influence market trends.
44. However, popular culture is not without its criticisms, as some argue that it can perpetuate harmful stereotypes, promote consumerism, and prioritize profit over social responsibility. It is important for consumers to be critical of the messages and values promoted in popular culture and to advocate for more diverse and inclusive representations.
45. Overall, popular culture is a complex and multifaceted phenomenon that reflects and influences our society in a variety of ways. By engaging with popular culture thoughtfully and consciously, we can harness its potential to inspire, educate, and empower individuals to create positive change in the world.


46. It is essential for individuals to recognize the power that popular culture holds and to be mindful of the messages it conveys. By consuming media and engaging with popular culture in a critical and discerning manner, we can challenge harmful narratives, support positive representations, and contribute to a more inclusive and equitable society.
47. Furthermore, popular culture has the ability to bring people together, fostering a sense of community and shared experience. Whether through attending concerts, participating in fan communities, or discussing the latest trends on social media, popular culture has the capacity to create connections and build relationships among individuals from diverse backgrounds.
48. As technology continues to evolve and new forms of media emerge, the landscape of popular culture is constantly shifting and expanding. It is important for individuals to stay informed about current trends and developments in order to fully engage with and understand the complexities of popular culture in today's society.
49. Ultimately, popular culture plays a crucial role in shaping our perceptions, attitudes, and behaviors. By actively participating in and contributing to popular culture in a thoughtful and intentional manner, we can harness its potential to inspire positive change and foster a more inclusive and accepting world for all.

50. In conclusion, popular culture is a powerful force that influences our daily lives in numerous ways. By being thoughtful and critical consumers of media, actively participating in cultural discussions, and promoting positive representations, we can harness the potential of popular culture to create a more diverse, inclusive, and equitable society for all. Let us embrace the opportunities that popular culture presents to us and work towards a future where everyone's voices and experiences are valued and celebrated.

Together, we can shape popular culture into a force for good, breaking down barriers and challenging stereotypes. Let us use the power of popular culture to inspire change, promote understanding, and create a more just and compassionate world. By working together, we can make a difference and bring about a more inclusive and equitable society for all. Let us continue to engage with and critique popular culture, using it as a tool for positive social change. The possibilities are endless, and the potential for impact is vast. Let us seize this opportunity and make a difference in the world through the power of popular culture.

In [44]:
completions[4]

{'id': 'chatcmpl-9rSyNew3bRbcf5FoSt88NJ9iu4ljW',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Some categories I can assist with include (but are not limited to):\n- General knowledge\n- Science\n- Technology\n- History\n- Health\n- Literature\n- Geography\n- Mathematics\n- Language and linguistics\n\nFeel free to ask me anything within these categories or any other topics you may be curious about!',
    'role': 'assistant',
    'function_call': None,
    'tool_calls': None}}],
 'created': 1722529783,
 'model': 'gpt-3.5-turbo-0125',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 64, 'prompt_tokens': 43, 'total_tokens': 107}}